# PhysioNet EEG-ARNN Training Pipeline

This notebook implements the complete training pipeline with:
- Subject-specific 3-fold cross-validation
- Edge Selection (ES) and Aggregation Selection (AS)
- Experiments with different k values (10, 15, 20, 25)
- Comprehensive results and visualizations

## Setup and Imports

In [9]:
import sys
from pathlib import Path
import warnings
import json
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import mne

from models import EEGARNN, ChannelSelector
from train_utils import (
    load_preprocessed_data, filter_classes, normalize_data,
    cross_validate_subject, EEGDataset
)

warnings.filterwarnings('ignore')
mne.set_log_level('ERROR')
sns.set_context('notebook', font_scale=1.1)
plt.style.use('seaborn-v0_8')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Using device: cpu


## Configuration

In [10]:
EXPERIMENT_CONFIG = {
    'data': {
        'preprocessed_dir': Path('data/physionet/derived/preprocessed'),
        'index_file': Path('data/physionet/derived/physionet_preprocessed_index.csv'),
        'selected_classes': [1, 2],  # T1, T2 (or [1, 2, 3, 4] for 4-class)
        'tmin': -1.0,
        'tmax': 5.0,
        'baseline': (-0.5, 0)
    },
    'model': {
        'hidden_dim': 40,
        'epochs': 50,  # Reduced from 100 to 50
        'learning_rate': 0.001,
        'batch_size': 32,
        'n_folds': 3
    },
    'channel_selection': {
        'k_values': [10, 15, 20, 25, 'all'],  # Different k to test
        'methods': ['ES', 'AS']  # Edge Selection, Aggregation Selection
    },
    'output': {
        'results_dir': Path('results'),
        'models_dir': Path('saved_models')
    },
    'max_subjects': 20  # Train on only 20 subjects
}

EXPERIMENT_CONFIG['output']['results_dir'].mkdir(exist_ok=True)
EXPERIMENT_CONFIG['output']['models_dir'].mkdir(exist_ok=True)

print("Experiment Configuration:")
print(json.dumps(EXPERIMENT_CONFIG, indent=2, default=str))

Experiment Configuration:
{
  "data": {
    "preprocessed_dir": "data\\physionet\\derived\\preprocessed",
    "index_file": "data\\physionet\\derived\\physionet_preprocessed_index.csv",
    "selected_classes": [
      1,
      2
    ],
    "tmin": -1.0,
    "tmax": 5.0,
    "baseline": [
      -0.5,
      0
    ]
  },
  "model": {
    "hidden_dim": 40,
    "epochs": 50,
    "learning_rate": 0.001,
    "batch_size": 32,
    "n_folds": 3
  },
  "channel_selection": {
    "k_values": [
      10,
      15,
      20,
      25,
      "all"
    ],
    "methods": [
      "ES",
      "AS"
    ]
  },
  "output": {
    "results_dir": "results",
    "models_dir": "saved_models"
  },
  "max_subjects": 20
}


## Load Preprocessed Data Index

In [11]:
index_df = pd.read_csv(EXPERIMENT_CONFIG['data']['index_file'])
success_df = index_df[index_df['status'] == 'success'].copy()

print(f"Total preprocessed runs: {len(index_df)}")
print(f"Successful runs: {len(success_df)}")
print(f"\nSubjects available: {success_df['subject'].nunique()}")
print(f"\nRuns per category:")
print(success_df.groupby('category')['run'].count())

success_df.head()

Total preprocessed runs: 669
Successful runs: 669

Subjects available: 51

Runs per category:
category
motor_execution    292
motor_imagery      295
resting_state       82
Name: run, dtype: int64


,subject,run,status,path,timestamp,original_sfreq,n_channels_original,duration_s,bad_channels,n_bad_channels,notch_applied,final_sfreq,n_channels_final,file_size_mb,category,task,bad_channels_interpolated
0,S001,R01,success,data\physionet\derived\preprocessed\S001\S001R...,2025-10-23T17:52:38.863386,160.0,64,60.99375,[],0,[50.0],128.0,64,1.917076,resting_state,Baseline eyes open,NaN
1,S001,R02,success,data\physionet\derived\preprocessed\S001\S001R...,2025-10-23T17:52:39.482853,160.0,64,60.99375,[],0,[50.0],128.0,64,1.917076,resting_state,Baseline eyes closed,NaN
2,S001,R04,success,data\physionet\derived\preprocessed\S001\S001R...,2025-10-23T17:52:39.869856,160.0,64,124.99375,[],0,[50.0],128.0,64,3.918468,motor_execution,Open/close right fist (executed),NaN
3,S001,R05,success,data\physionet\derived\preprocessed\S001\S001R...,2025-10-23T17:52:40.658595,160.0,64,124.99375,[],0,[50.0],128.0,64,3.918468,motor_execution,Open/close both fists (executed),NaN
4,S001,R06,success,data\physionet\derived\preprocessed\S001\S001R...,2025-10-23T17:52:41.551311,160.0,64,124.99375,[],0,[50.0],128.0,64,3.918468,motor_execution,Open/close both feet (executed),NaN


## Subject Selection

Select subjects for training. For motor imagery, we focus on motor execution and motor imagery runs.

In [12]:
# Filter motor-related runs (execution and imagery)
motor_runs = success_df[success_df['category'].isin(['motor_execution', 'motor_imagery'])].copy()

# Count runs per subject
subject_counts = motor_runs.groupby('subject').size().reset_index(name='num_runs')
subject_counts = subject_counts[subject_counts['num_runs'] >= 10]  # At least 10 runs

selected_subjects = subject_counts['subject'].tolist()

print(f"Subjects with >=10 motor runs: {len(selected_subjects)}")
print(f"\nFirst 10 subjects: {selected_subjects[:10]}")

# Limit to max_subjects for this experiment
max_subjects = EXPERIMENT_CONFIG.get('max_subjects', len(selected_subjects))
selected_subjects = selected_subjects[:max_subjects]

print(f"\nWill train on {len(selected_subjects)} subjects")
print(f"Selected subjects: {selected_subjects}")

Subjects with >=10 motor runs: 49

First 10 subjects: ['S001', 'S002', 'S005', 'S006', 'S007', 'S008', 'S011', 'S014', 'S015', 'S016']

Will train on 20 subjects
Selected subjects: ['S001', 'S002', 'S005', 'S006', 'S007', 'S008', 'S011', 'S014', 'S015', 'S016', 'S018', 'S020', 'S025', 'S026', 'S029', 'S030', 'S033', 'S034', 'S035', 'S037']


## Helper Functions

In [13]:
def load_subject_data(subject_id, preprocessed_dir, motor_runs_df, config):
    """
    Load all motor runs for a subject and concatenate
    
    Returns
    -------
    data : np.ndarray or None
        (n_trials, n_channels, n_timepoints)
    labels : np.ndarray or None
        (n_trials,)
    channel_names : list
        List of channel names
    """
    subject_runs = motor_runs_df[motor_runs_df['subject'] == subject_id]
    
    all_data = []
    all_labels = []
    channel_names = None
    
    for _, run_info in subject_runs.iterrows():
        fif_path = Path(run_info['path'])
        
        if not fif_path.exists():
            continue
        
        try:
            data, labels = load_preprocessed_data(
                fif_path,
                tmin=config['data']['tmin'],
                tmax=config['data']['tmax'],
                baseline=config['data']['baseline']
            )
            
            if data is not None and len(data) > 0:
                all_data.append(data)
                all_labels.append(labels)
                
                if channel_names is None:
                    raw = mne.io.read_raw_fif(fif_path, preload=False, verbose='ERROR')
                    channel_names = raw.ch_names
                    
        except Exception as e:
            print(f"Error loading {fif_path.name}: {e}")
            continue
    
    if len(all_data) == 0:
        return None, None, None
    
    all_data = np.concatenate(all_data, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    
    # Filter to selected classes
    all_data, all_labels = filter_classes(
        all_data, all_labels, config['data']['selected_classes']
    )
    
    return all_data, all_labels, channel_names

## Main Training Loop

Train subject-specific models with 3-fold cross-validation

In [14]:
all_results = []

for subject_id in tqdm(selected_subjects, desc="Training subjects"):
    print(f"\n{'='*80}")
    print(f"Training subject: {subject_id}")
    print(f"{'='*80}")
    
    # Load subject data
    data, labels, channel_names = load_subject_data(
        subject_id, 
        EXPERIMENT_CONFIG['data']['preprocessed_dir'],
        motor_runs,
        EXPERIMENT_CONFIG
    )
    
    if data is None or len(data) < 30:  # Need at least 30 trials for 3-fold CV
        print(f"Skipping {subject_id}: insufficient data")
        continue
    
    print(f"Data shape: {data.shape}")
    print(f"Labels: {np.unique(labels, return_counts=True)}")
    print(f"Channels: {len(channel_names)}")
    
    num_channels = data.shape[1]
    num_timepoints = data.shape[2]
    num_classes = len(np.unique(labels))
    
    # Perform 3-fold cross-validation
    cv_results = cross_validate_subject(
        data, labels,
        num_channels=num_channels,
        num_timepoints=num_timepoints,
        num_classes=num_classes,
        device=device,
        n_splits=EXPERIMENT_CONFIG['model']['n_folds'],
        epochs=EXPERIMENT_CONFIG['model']['epochs'],
        lr=EXPERIMENT_CONFIG['model']['learning_rate']
    )
    
    print(f"\nAverage accuracy (all channels): {cv_results['avg_accuracy']:.4f} ± {cv_results['std_accuracy']:.4f}")
    
    # Save results
    result = {
        'subject': subject_id,
        'num_trials': len(data),
        'num_channels': num_channels,
        'num_timepoints': num_timepoints,
        'num_classes': num_classes,
        'all_channels_acc': cv_results['avg_accuracy'],
        'all_channels_std': cv_results['std_accuracy'],
        'adjacency_matrix': cv_results['adjacency_matrix'],
        'channel_names': channel_names
    }
    
    all_results.append(result)

print(f"\n{'='*80}")
print(f"Training complete for {len(all_results)} subjects")
print(f"{'='*80}")

Training subjects:   0%|          | 0/20 [00:00<?, ?it/s]


Training subject: S001
Error loading S001R04_preproc_raw.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Error loading S001R05_preproc_raw.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Error loading S001R06_preproc_raw.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Error loading S001R07_preproc_raw.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Error loading S001R08_preproc_raw.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Error loading S001R09_preproc_raw.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_f

Training subjects:   5%|▌         | 1/20 [00:00<00:09,  1.92it/s]

Error loading S001R12_preproc_raw.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Error loading S001R13_preproc_raw.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Error loading S001R14_preproc_raw.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Skipping S001: insufficient data

Training subject: S002
Error loading S002R03_preproc_raw.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Error loading S002R04_preproc_raw.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Error loading S002R05_preproc_raw.fif: No stim channels found, but the raw object has annotat

Training subjects:  10%|█         | 2/20 [00:01<00:09,  1.85it/s]

...

Error loading S034R12_preproc_raw.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Error loading S034R13_preproc_raw.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Error loading S034R14_preproc_raw.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Skipping S034: insufficient data

Training subject: S035
Error loading S035R03_preproc_raw.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Error loading S035R04_preproc_raw.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Error loading S035R05_preproc_raw.fif: No stim channels found, but the raw object has annotat

Training subjects:  95%|█████████▌| 19/20 [00:12<00:00,  1.43it/s]

Error loading S035R13_preproc_raw.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Error loading S035R14_preproc_raw.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Skipping S035: insufficient data

Training subject: S037
Error loading S037R03_preproc_raw.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Error loading S037R04_preproc_raw.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Error loading S037R05_preproc_raw.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Error loading S037R06_preproc_raw.fif: No stim channels found, but the raw object has annotat

Training subjects: 100%|██████████| 20/20 [00:12<00:00,  1.56it/s]

Error loading S037R12_preproc_raw.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Error loading S037R13_preproc_raw.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Error loading S037R14_preproc_raw.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Skipping S037: insufficient data

Training complete for 0 subjects


## Channel Selection Experiments

Test different k values with Edge Selection and Aggregation Selection

In [15]:
channel_selection_results = []

if len(all_results) > 0:
    for result in tqdm(all_results, desc="Channel selection experiments"):
        subject_id = result['subject']
        adj_matrix = result['adjacency_matrix']
        channel_names = result['channel_names']
        
        print(f"\nProcessing channel selection for {subject_id}")
        
        selector = ChannelSelector(adj_matrix, channel_names)
        
        for method in EXPERIMENT_CONFIG['channel_selection']['methods']:
            print(f"  Method: {method}")
            
            for k in EXPERIMENT_CONFIG['channel_selection']['k_values']:
                if k == 'all':
                    k_val = result['num_channels']
                    selected_channels = channel_names
                else:
                    k_val = min(k, result['num_channels'])  # Don't exceed available channels
                    
                    if method == 'ES':
                        selected_channels, _ = selector.edge_selection(k_val)
                    else:  # AS
                        selected_channels, _ = selector.aggregation_selection(k_val)
                
                print(f"    k={k_val}: {len(selected_channels)} channels selected")
                
                channel_selection_results.append({
                    'subject': subject_id,
                    'method': method,
                    'k': k_val,
                    'num_selected': len(selected_channels),
                    'selected_channels': selected_channels,
                    'accuracy_full': result['all_channels_acc']
                })

    channel_selection_df = pd.DataFrame(channel_selection_results)
    print(f"\nChannel selection results: {len(channel_selection_df)} experiments")
    display(channel_selection_df.head(10))
else:
    channel_selection_df = pd.DataFrame()
    print("\nNo results available for channel selection experiments.")


No results available for channel selection experiments.


## Results Summary

In [16]:
results_df = pd.DataFrame(all_results)

print("=" * 80)
print("OVERALL RESULTS SUMMARY")
print("=" * 80)
print(f"\nSubjects trained: {len(results_df)}")

if len(results_df) > 0:
    print(f"Mean accuracy (all channels): {results_df['all_channels_acc'].mean():.4f} ± {results_df['all_channels_acc'].std():.4f}")
    print(f"Best subject: {results_df.loc[results_df['all_channels_acc'].idxmax(), 'subject']} ({results_df['all_channels_acc'].max():.4f})")
    print(f"Worst subject: {results_df.loc[results_df['all_channels_acc'].idxmin(), 'subject']} ({results_df['all_channels_acc'].min():.4f})")

    # Save results
    results_path = EXPERIMENT_CONFIG['output']['results_dir'] / 'subject_results.csv'
    results_df[['subject', 'num_trials', 'num_channels', 'all_channels_acc', 'all_channels_std']].to_csv(results_path, index=False)
    print(f"\nResults saved to: {results_path}")

    display(results_df[['subject', 'num_trials', 'num_channels', 'all_channels_acc', 'all_channels_std']].head(10))
else:
    print("\nNo subjects were successfully trained. Check the data loading and preprocessing steps.")
    results_df

OVERALL RESULTS SUMMARY

Subjects trained: 0

No subjects were successfully trained. Check the data loading and preprocessing steps.


## Visualizations

In [17]:
if len(results_df) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))

    # Accuracy distribution
    axes[0, 0].hist(results_df['all_channels_acc'], bins=20, color='steelblue', edgecolor='black', alpha=0.7)
    axes[0, 0].axvline(results_df['all_channels_acc'].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
    axes[0, 0].set_title('Accuracy Distribution (All Channels)', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Accuracy')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # Accuracy vs num trials
    axes[0, 1].scatter(results_df['num_trials'], results_df['all_channels_acc'], alpha=0.6, s=100)
    axes[0, 1].set_title('Accuracy vs Number of Trials', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Number of Trials')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].grid(True, alpha=0.3)

    # Top 10 subjects
    top_10 = results_df.nlargest(min(10, len(results_df)), 'all_channels_acc')
    axes[1, 0].barh(range(len(top_10)), top_10['all_channels_acc'], color='green', alpha=0.7)
    axes[1, 0].set_yticks(range(len(top_10)))
    axes[1, 0].set_yticklabels(top_10['subject'])
    axes[1, 0].set_title(f'Top {len(top_10)} Subjects by Accuracy', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Accuracy')
    axes[1, 0].invert_yaxis()
    axes[1, 0].grid(True, alpha=0.3, axis='x')

    # Subject ranking
    sorted_results = results_df.sort_values('all_channels_acc')
    axes[1, 1].plot(range(len(sorted_results)), sorted_results['all_channels_acc'], marker='o', markersize=4, alpha=0.6)
    axes[1, 1].set_title('Subject Ranking', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Rank')
    axes[1, 1].set_ylabel('Accuracy')
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(EXPERIMENT_CONFIG['output']['results_dir'] / 'results_summary.png', dpi=300, bbox_inches='tight')
    plt.show()

    print(f"Visualizations saved to: {EXPERIMENT_CONFIG['output']['results_dir']}")
else:
    print("No results to visualize. Please ensure subjects were successfully trained.")

No results to visualize. Please ensure subjects were successfully trained.


## Visualize Learned Adjacency Matrix (Example Subject)

In [18]:
if len(all_results) > 0:
    # Pick best subject
    best_idx = results_df['all_channels_acc'].idxmax()
    best_result = all_results[best_idx]
    
    print(f"Visualizing adjacency matrix for best subject: {best_result['subject']}")
    print(f"Accuracy: {best_result['all_channels_acc']:.4f}")
    
    selector = ChannelSelector(best_result['adjacency_matrix'], best_result['channel_names'])
    
    fig = selector.visualize_adjacency(
        save_path=EXPERIMENT_CONFIG['output']['results_dir'] / f"adjacency_{best_result['subject']}.png"
    )
    plt.show()
    
    # Show top edges
    print("\nTop 10 Edges (Edge Selection):")
    selected_channels_es, _ = selector.edge_selection(10)
    print(f"Selected channels: {selected_channels_es}")
    
    print("\nTop 10 Channels (Aggregation Selection):")
    selected_channels_as, _ = selector.aggregation_selection(10)
    print(f"Selected channels: {selected_channels_as}")

## Export Results

In [19]:
if len(results_df) > 0:
    # Save subject results
    results_path = EXPERIMENT_CONFIG['output']['results_dir'] / 'subject_results.csv'
    results_df[['subject', 'num_trials', 'num_channels', 'all_channels_acc', 'all_channels_std']].to_csv(results_path, index=False)
    
    # Save channel selection results
    if len(channel_selection_df) > 0:
        channel_selection_path = EXPERIMENT_CONFIG['output']['results_dir'] / 'channel_selection_results.csv'
        channel_selection_df.to_csv(channel_selection_path, index=False)
    else:
        channel_selection_path = None
    
    # Save experiment config
    config_path = EXPERIMENT_CONFIG['output']['results_dir'] / 'experiment_config.json'
    with open(config_path, 'w') as f:
        json.dump(EXPERIMENT_CONFIG, f, indent=2, default=str)
    
    print("All results exported successfully!")
    print(f"  - Subject results: {results_path}")
    if channel_selection_path:
        print(f"  - Channel selection: {channel_selection_path}")
    print(f"  - Config: {config_path}")
else:
    print("No results to export.")

No results to export.
